# CMA and DEMA Algorithms for Micro-Aethalometer Data

## 1. Introduction and Background

This notebook implements two post-processing algorithms for micro-aethalometer data:
 
1. **Centered Moving Average (CMA)**: A smoothing technique that incorporates data points both before and after each measurement to reduce noise while preserving microenvironmental characteristics.

2. **Double Exponentially Weighted Moving Average (DEMA)**: A smoothing approach that reduces noise-induced artifacts while limiting lag, especially useful for source apportionment calculations.

Both of these methods have been shown to outperform the Optimized Noise-reduction Algorithm (ONA) for newer dual-spot aethalometers, as demonstrated in recent research by Liu et al. (2021) and Mendoza et al. (2024).

## 2. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import seaborn as sns
from IPython.display import display

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 3. Load and Explore the Data

First, let's load the Aethalometer data and examine its structure.

In [ ]:
# Define the file path - replace with your actual file path
file_path = "aethalometer_data.csv"

# Load the data
data = pd.read_csv(file_path)

# Display basic information about the dataset
print(f"Dataset shape: {data.shape}")
print("\nColumn names:")
print(data.columns.tolist())

# Display the first few rows
print("\nFirst few rows of the dataset:")
display(data.head())

# Check for the presence of BC columns for each wavelength
wavelengths = ['UV', 'Blue', 'Green', 'Red', 'IR']
for wavelength in wavelengths:
    bc_col = f"{wavelength} BC1"
    
    if bc_col in data.columns:
        print(f"\n{wavelength} wavelength data:")
        print(f"  BC range: {data[bc_col].min()} to {data[bc_col].max()} ng/m³")
        print(f"  Negative BC values: {(data[bc_col] < 0).sum()} ({(data[bc_col] < 0).sum() / len(data) * 100:.2f}%)")
    else:
        print(f"\nWarning: {wavelength} data columns not found")

# Check the time resolution
if 'Timebase (s)' in data.columns:
    timebase = data['Timebase (s)'].iloc[0]
    print(f"\nInstrument timebase: {timebase} seconds")
else:
    print("\nTimebase column not found")


## 4. CMA Algorithm Implementation
 
The Centered Moving Average is a smoothing technique that uses data points both before and after each measurement. This helps maintain the central trend without introducing lag or phase shifts.

In [ ]:
def apply_cma(data, wavelength='Blue', window_size=None):
    """
    Apply the Centered Moving Average algorithm to Aethalometer data
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame containing Aethalometer data
    wavelength : str
        Which wavelength to process ('UV', 'Blue', 'Green', 'Red', 'IR')
    window_size : int or None
        Size of the moving average window (must be odd). If None, 
        will use a default based on the data's timebase
        
    Returns:
    --------
    data_smoothed : pandas.DataFrame
        DataFrame with the original data plus additional columns for smoothed BC
    """
    # Create a copy of the input dataframe
    data_smoothed = data.copy()
    
    # Identify the column for BC values based on wavelength
    bc_col = f"{wavelength} BC1"
    
    # Determine window size if not specified
    if window_size is None:
        if 'Timebase (s)' in data.columns:
            timebase = data['Timebase (s)'].iloc[0]
            if timebase == 1:
                window_size = 11  # 11 seconds for 1-second data
            elif timebase == 5:
                window_size = 5   # 25 seconds for 5-second data
            elif timebase == 60:
                window_size = 3   # 3 minutes for 1-minute data
            else:
                window_size = 5   # Default for other timebases
        else:
            window_size = 5       # Default if timebase is unknown
    
    # Make sure window_size is odd
    if window_size % 2 == 0:
        window_size += 1
    
    print(f"Using window size of {window_size} for CMA")
    
    # Add columns for CMA results
    smoothed_bc_col = f"{wavelength}_BC_CMA"
    data_smoothed[smoothed_bc_col] = data_smoothed[bc_col].rolling(
        window=window_size, center=True, min_periods=1
    ).mean()
    
    return data_smoothed

## 5. DEMA Algorithm Implementation

The Double Exponentially Weighted Moving Average applies additional smoothing to an EMA to reduce noise while limiting lag effects. It's particularly useful for source apportionment calculations.

In [ ]:
def apply_dema(data, wavelength='Blue', alpha=0.125):
    """
    Apply the Double Exponentially Weighted Moving Average algorithm to Aethalometer data
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame containing Aethalometer data
    wavelength : str
        Which wavelength to process ('UV', 'Blue', 'Green', 'Red', 'IR')
    alpha : float
        Smoothing parameter (between 0 and 1)
        For 60s data, 0.125 approximates a 15-minute smoothing window
        
    Returns:
    --------
    data_smoothed : pandas.DataFrame
        DataFrame with the original data plus additional columns for smoothed BC
    """
    # Create a copy of the input dataframe
    data_smoothed = data.copy()
    
    # Identify the column for BC values based on wavelength
    bc_col = f"{wavelength} BC1"
    
    # Set the smoothing parameter based on timebase if not explicitly provided
    if 'Timebase (s)' in data.columns:
        timebase = data['Timebase (s)'].iloc[0]
        if alpha is None:
            # Use formula 2/(N+1) where N is the desired smoothing period
            if timebase == 1:
                # Default to approximate 5-minute window for 1-second data
                N = 300 / timebase
            elif timebase == 5:
                # Default to approximate 5-minute window for 5-second data
                N = 300 / timebase
            elif timebase == 60:
                # Default to approximate 15-minute window for 60-second data
                N = 900 / timebase
            else:
                N = 15  # Default for other timebases
                
            alpha = 2 / (N + 1)
    
    print(f"Using alpha of {alpha:.4f} for DEMA")
    
    # Calculate EMA
    ema_col = f"{wavelength}_EMA"
    # First EMA
    data_smoothed[ema_col] = data_smoothed[bc_col].ewm(alpha=alpha, adjust=False).mean()
    
    # Calculate DEMA: (2 * EMA) - EMA(EMA)
    dema_col = f"{wavelength}_BC_DEMA"
    ema_of_ema = data_smoothed[ema_col].ewm(alpha=alpha, adjust=False).mean()
    data_smoothed[dema_col] = (2 * data_smoothed[ema_col]) - ema_of_ema
    
    return data_smoothed

## 6. Apply Processing Pipeline for Source Apportionment

Following the recommendations from recent research, we'll implement a complete processing pipeline:
1. Apply CMA to the raw BC data (both IR and Blue wavelengths)
2. Calculate source apportionment parameters
3. Apply DEMA to the source apportionment results
4. Limit BB% values to between 0 and 100%

In [ ]:
def calculate_source_apportionment(data, aae_wb=2.0, aae_ff=1.0):
    """
    Calculate source apportionment using the Aethalometer Model
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with BC data at blue and IR wavelengths
    aae_wb : float
        Absorption Ångström Exponent for wood burning (default 2.0)
    aae_ff : float
        Absorption Ångström Exponent for fossil fuel (default 1.0)
    
    Returns:
    --------
    data_sa : pandas.DataFrame
        DataFrame with additional source apportionment columns
    """
    # Create a copy of the input dataframe
    data_sa = data.copy()
    
    # Get BC values at blue and IR wavelengths (using CMA processed data if available)
    if 'Blue_BC_CMA' in data_sa.columns:
        bc_blue = data_sa['Blue_BC_CMA']
    else:
        bc_blue = data_sa['Blue BC1']
        
    if 'IR_BC_CMA' in data_sa.columns:
        bc_ir = data_sa['IR_BC_CMA']
    else:
        bc_ir = data_sa['IR BC1']
    
    # Calculate absorption coefficients
    # For simplicity, using approximate MACs and Cref values from literature
    mac_blue = 10.12  # m²/g at 470nm
    mac_ir = 7.77     # m²/g at 880nm
    c_ref = 1.3       # Multiple scattering enhancement factor
    
    babs_blue = bc_blue * mac_blue / c_ref
    babs_ir = bc_ir * mac_ir / c_ref
    
    # Calculate the ratio of wavelengths for the Aethalometer model
    wavelength_ratio = 470 / 880
    
    # Calculate absorption coefficients for wood burning and fossil fuel at IR wavelength
    # Using the Aethalometer model equations
    babs_ff_ir = (babs_blue - babs_ir * (wavelength_ratio ** (-aae_wb))) / \
                 ((wavelength_ratio ** (-aae_ff)) - (wavelength_ratio ** (-aae_wb)))
                 
    babs_wb_ir = babs_ir - babs_ff_ir
    
    # Calculate BB% (biomass burning percentage)
    bb_percent = 100 * babs_wb_ir / babs_ir
    
    # Handle infinity and NaN values
    bb_percent = bb_percent.replace([np.inf, -np.inf], np.nan)
    bb_percent = bb_percent.fillna(0)
    
    # Store results
    data_sa['Babs_Blue'] = babs_blue
    data_sa['Babs_IR'] = babs_ir
    data_sa['Babs_FF_IR'] = babs_ff_ir
    data_sa['Babs_WB_IR'] = babs_wb_ir
    data_sa['BB_Percent'] = bb_percent
    
    # Calculate BC from wood burning and fossil fuel
    data_sa['BC_WB'] = bc_ir * bb_percent / 100
    data_sa['BC_FF'] = bc_ir * (100 - bb_percent) / 100
    
    return data_sa


In [ ]:
def apply_dema_to_source_apportionment(data, alpha=0.125):
    """
    Apply DEMA specifically to source apportionment results
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with source apportionment data
    alpha : float
        Smoothing parameter
        
    Returns:
    --------
    data_smoothed : pandas.DataFrame
        DataFrame with smoothed source apportionment data
    """
    # Create a copy of the input dataframe
    data_smoothed = data.copy()
    
    # Apply DEMA to BB_Percent
    if 'BB_Percent' in data_smoothed.columns:
        # First EMA
        data_smoothed['BB_Percent_EMA'] = data_smoothed['BB_Percent'].ewm(alpha=alpha, adjust=False).mean()
        
        # Calculate DEMA
        ema_of_ema = data_smoothed['BB_Percent_EMA'].ewm(alpha=alpha, adjust=False).mean()
        data_smoothed['BB_Percent_DEMA'] = (2 * data_smoothed['BB_Percent_EMA']) - ema_of_ema
        
        # Limit BB% to logical range [0, 100]
        data_smoothed['BB_Percent_DEMA'] = data_smoothed['BB_Percent_DEMA'].clip(0, 100)
        
        # Recalculate BC_WB and BC_FF using smoothed BB_Percent
        if 'IR_BC_CMA' in data_smoothed.columns:
            bc_ir = data_smoothed['IR_BC_CMA']
        else:
            bc_ir = data_smoothed['IR BC1']
            
        data_smoothed['BC_WB_DEMA'] = bc_ir * data_smoothed['BB_Percent_DEMA'] / 100
        data_smoothed['BC_FF_DEMA'] = bc_ir * (100 - data_smoothed['BB_Percent_DEMA']) / 100
    
    return data_smoothed

## 7. Apply Full Processing Pipeline to Data

In [ ]:
# Step 1: Apply CMA to Blue and IR BC data
processed_data = data.copy()
for wavelength in ['Blue', 'IR']:
    bc_col = f"{wavelength} BC1"
    if bc_col in processed_data.columns:
        print(f"\nApplying CMA to {wavelength} wavelength data...")
        processed_data = apply_cma(processed_data, wavelength)

# Step 2: Calculate source apportionment
print("\nCalculating source apportionment...")
processed_data = calculate_source_apportionment(processed_data)

# Step 3: Apply DEMA to source apportionment results
print("\nApplying DEMA to source apportionment results...")
processed_data = apply_dema_to_source_apportionment(processed_data)

## 8. Evaluate Processing Performance
 
Now let's evaluate how well our processing methods performed.

In [ ]:
def evaluate_processing(data, raw_col, cma_col, dema_col=None):
    """
    Evaluate the performance of the processing algorithms
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with original and processed data
    raw_col : str
        Column name for raw data
    cma_col : str
        Column name for CMA processed data
    dema_col : str, optional
        Column name for DEMA processed data (if applicable)
    """
    # 1. Reduction of negatives
    numneg_raw = (data[raw_col] < 0).sum() / len(data)
    numneg_cma = (data[cma_col] < 0).sum() / len(data)
    
    print(f"Fraction of negative values in raw data: {numneg_raw:.4f}")
    print(f"Fraction of negative values after CMA: {numneg_cma:.4f}")
    
    if dema_col is not None and dema_col in data.columns:
        numneg_dema = (data[dema_col] < 0).sum() / len(data)
        print(f"Fraction of negative values after DEMA: {numneg_dema:.4f}")
    
    # 2. Reduction of noise (average absolute difference between consecutive points)
    temp_raw = np.zeros(len(data)-1)
    temp_cma = np.zeros(len(data)-1)
    
    for i in range(len(data)-1):
        temp_raw[i] = abs(data[raw_col].iloc[i+1] - data[raw_col].iloc[i])
        temp_cma[i] = abs(data[cma_col].iloc[i+1] - data[cma_col].iloc[i])
    
    noise_raw = np.nanmean(temp_raw)
    noise_cma = np.nanmean(temp_cma)
    
    print(f"Noise in raw data: {noise_raw:.1f} ng/m³")
    print(f"Noise in CMA data: {noise_cma:.1f} ng/m³")
    print(f"Noise reduction factor with CMA: {noise_raw/noise_cma:.1f}x")
    
    if dema_col is not None and dema_col in data.columns:
        temp_dema = np.zeros(len(data)-1)
        for i in range(len(data)-1):
            temp_dema[i] = abs(data[dema_col].iloc[i+1] - data[dema_col].iloc[i])
        noise_dema = np.nanmean(temp_dema)
        print(f"Noise in DEMA data: {noise_dema:.1f} ng/m³")
        print(f"Noise reduction factor with DEMA: {noise_raw/noise_dema:.1f}x")

# Evaluate each wavelength
for wavelength in ['Blue', 'IR']:
    bc_col = f"{wavelength} BC1"
    cma_col = f"{wavelength}_BC_CMA"
    
    if bc_col in processed_data.columns and cma_col in processed_data.columns:
        print(f"\nEvaluating {wavelength} wavelength:")
        evaluate_processing(processed_data, bc_col, cma_col)

# Evaluate source apportionment processing
if 'BB_Percent' in processed_data.columns and 'BB_Percent_DEMA' in processed_data.columns:
    print("\nEvaluating source apportionment processing:")
    evaluate_processing(processed_data, 'BB_Percent', 'BB_Percent', 'BB_Percent_DEMA')

## 9. Visualize Results
 
Let's visualize the raw and processed data to see the effects of our algorithms.

In [ ]:
def plot_results(data, wavelength, sample_period=None):
    """
    Plot the raw, CMA, and DEMA processed BC data
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with raw and processed data
    wavelength : str
        Which wavelength to plot
    sample_period : tuple, optional
        Start and end indices for a subset of the data to plot
    """
    # Identify columns
    bc_col = f"{wavelength} BC1"
    cma_col = f"{wavelength}_BC_CMA"
    
    # Select a subset of data if specified
    if sample_period is not None:
        start_idx, end_idx = sample_period
        plot_data = data.iloc[start_idx:end_idx].copy()
    else:
        plot_data = data.copy()
    
    # Create a figure
    plt.figure(figsize=(12, 6))
    
    # Create x-axis values
    if 'Time (UTC)' in plot_data.columns:
        try:
            x = pd.to_datetime(plot_data['Time (UTC)'])
            x_formatter = mdates.DateFormatter('%H:%M')
            plt.gca().xaxis.set_major_formatter(x_formatter)
            plt.gcf().autofmt_xdate()
            x_label = 'Time (UTC)'
        except:
            x = np.arange(len(plot_data))
            x_label = 'Data Point'
    else:
        x = np.arange(len(plot_data))
        x_label = 'Data Point'
    
    # Plot BC data
    plt.plot(x, plot_data[bc_col], 'k-', alpha=0.5, label='Raw Data')
    plt.plot(x, plot_data[cma_col], 'r-', label='CMA')
    
    plt.xlabel(x_label)
    plt.ylabel(f'{wavelength} BC (ng/m³)')
    plt.title(f'CMA Performance for {wavelength} Wavelength')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_source_apportionment(data, sample_period=None):
    """
    Plot the source apportionment results (raw and processed)
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with raw and processed source apportionment data
    sample_period : tuple, optional
        Start and end indices for a subset of the data to plot
    """
    # Select columns for source apportionment
    cols = ['BB_Percent', 'BB_Percent_DEMA', 'BC_WB', 'BC_WB_DEMA', 'BC_FF', 'BC_FF_DEMA']
    
    # Check if all needed columns exist
    if not all(col in data.columns for col in cols):
        print("Missing some source apportionment columns")
        return
    
    # Select a subset of data if specified
    if sample_period is not None:
        start_idx, end_idx = sample_period
        plot_data = data.iloc[start_idx:end_idx].copy()
    else:
        plot_data = data.copy()
    
    # Create x-axis values
    if 'Time (UTC)' in plot_data.columns:
        try:
            x = pd.to_datetime(plot_data['Time (UTC)'])
            x_formatter = mdates.DateFormatter('%H:%M')
            x_label = 'Time (UTC)'
        except:
            x = np.arange(len(plot_data))
            x_label = 'Data Point'
    else:
        x = np.arange(len(plot_data))
        x_label = 'Data Point'
    
    # Create a figure with three subplots
    fig, axes = plt.subplots(3, 1, figsize=(12, 12), sharex=True)
    
    # Plot BB percentage
    axes[0].plot(x, plot_data['BB_Percent'], 'k-', alpha=0.5, label='Raw')
    axes[0].plot(x, plot_data['BB_Percent_DEMA'], 'g-', label='DEMA')
    axes[0].set_ylabel('Biomass Burning %')
    axes[0].set_title('Source Apportionment Results')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot Wood Burning BC
    axes[1].plot(x, plot_data['BC_WB'], 'k-', alpha=0.5, label='Raw')
    axes[1].plot(x, plot_data['BC_WB_DEMA'], 'g-', label='DEMA')
    axes[1].set_ylabel('Wood Burning BC (ng/m³)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Plot Fossil Fuel BC
    axes[2].plot(x, plot_data['BC_FF'], 'k-', alpha=0.5, label='Raw')
    axes[2].plot(x, plot_data['BC_FF_DEMA'], 'g-', label='DEMA')
    axes[2].set_ylabel('Fossil Fuel BC (ng/m³)')
    axes[2].set_xlabel(x_label)
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    if x_label == 'Time (UTC)':
        for ax in axes:
            ax.xaxis.set_major_formatter(x_formatter)
        fig.autofmt_xdate()
    
    plt.tight_layout()
    plt.show()

# Plot results for each wavelength
for wavelength in ['Blue', 'IR']:
    bc_col = f"{wavelength} BC1"
    cma_col = f"{wavelength}_BC_CMA"
    
    if bc_col in processed_data.columns and cma_col in processed_data.columns:
        print(f"\nPlots for {wavelength} wavelength:")
        
        # Plot full dataset
        plot_results(processed_data, wavelength)
        
        # Plot a sample period (first 1000 points or 10% of data, whichever is smaller)
        sample_size = min(1000, int(len(processed_data) * 0.1))
        if sample_size < len(processed_data):
            print(f"\nZoomed view of first {sample_size} points:")
            plot_results(processed_data, wavelength, (0, sample_size))

# Plot source apportionment results
if all(col in processed_data.columns for col in ['BB_Percent', 'BB_Percent_DEMA']):
    print("\nSource apportionment plots:")
    
    # Plot full dataset
    plot_source_apportionment(processed_data)
    
    # Plot a sample period
    sample_size = min(1000, int(len(processed_data) * 0.1))
    if sample_size < len(processed_data):
        print(f"\nZoomed view of first {sample_size} points:")
        plot_source_apportionment(processed_data, (0, sample_size))

## 10. Compare CMA with ONA (if available)

If ONA processed data is available, let's compare it with the CMA results.

In [ ]:
def compare_cma_ona(data, wavelength='Blue'):
    """
    Compare CMA with ONA results if available
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with CMA and possibly ONA data
    wavelength : str
        Which wavelength to compare
    """
    bc_col = f"{wavelength} BC1"
    cma_col = f"{wavelength}_BC_CMA"
    ona_col = f"{wavelength}_BC_smoothed"  # Expected column name from ONA algorithm
    
    if bc_col in data.columns and cma_col in data.columns and ona_col in data.columns:
        print(f"\nComparing CMA and ONA for {wavelength} wavelength:")
        
        # Calculate metrics for both algorithms
        # 1. Reduction of negatives
        numneg_raw = (data[bc_col] < 0).sum() / len(data)
        numneg_cma = (data[cma_col] < 0).sum() / len(data)
        numneg_ona = (data[ona_col] < 0).sum() / len(data)
        
        print(f"Fraction of negative values in raw data: {numneg_raw:.4f}")
        print(f"Fraction of negative values after CMA: {numneg_cma:.4f}")
        print(f"Fraction of negative values after ONA: {numneg_ona:.4f}")
        
        # 2. Reduction of noise
        temp_raw = np.zeros(len(data)-1)
        temp_cma = np.zeros(len(data)-1)
        temp_ona = np.zeros(len(data)-1)
        
        for i in range(len(data)-1):
            temp_raw[i] = abs(data[bc_col].iloc[i+1] - data[bc_col].iloc[i])
            temp_cma[i] = abs(data[cma_col].iloc[i+1] - data[cma_col].iloc[i])
            temp_ona[i] = abs(data[ona_col].iloc[i+1] - data[ona_col].iloc[i])
        
        noise_raw = np.nanmean(temp_raw)
        noise_cma = np.nanmean(temp_cma)
        noise_ona = np.nanmean(temp_ona)
        
        print(f"Noise in raw data: {noise_raw:.1f} ng/m³")
        print(f"Noise in CMA data: {noise_cma:.1f} ng/m³")
        print(f"Noise in ONA data: {noise_ona:.1f} ng/m³")
        print(f"Noise reduction factor with CMA: {noise_raw/noise_cma:.1f}x")
        print(f"Noise reduction factor with ONA: {noise_raw/noise_ona:.1f}x")
        
        # 3. Plot comparison
        # Select a subset of data for clarity (first 1000 points or 10% of data)
        sample_size = min(1000, int(len(data) * 0.1))
        plot_data = data.iloc[:sample_size].copy()
        
        plt.figure(figsize=(12, 6))
        
        # Create x-axis
        if 'Time (UTC)' in plot_data.columns:
            try:
                x = pd.to_datetime(plot_data['Time (UTC)'])
                x_formatter = mdates.DateFormatter('%H:%M')
                plt.gca().xaxis.set_major_formatter(x_formatter)
                plt.gcf().autofmt_xdate()
                x_label = 'Time (UTC)'
            except:
                x = np.arange(len(plot_data))
                x_label = 'Data Point'
        else:
            x = np.arange(len(plot_data))
            x_label = 'Data Point'
        
        # Plot data
        plt.plot(x, plot_data[bc_col], 'k-', alpha=0.3, label='Raw Data')
        plt.plot(x, plot_data[cma_col], 'r-', label='CMA')
        plt.plot(x, plot_data[ona_col], 'b-', label='ONA')
        
        plt.xlabel(x_label)
        plt.ylabel(f'{wavelength} BC (ng/m³)')
        plt.title(f'Comparison of CMA and ONA for {wavelength} Wavelength')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Create a histogram of differences between CMA and ONA
        plt.figure(figsize=(10, 6))
        diff = data[cma_col] - data[ona_col]
        plt.hist(diff, bins=50)
        plt.xlabel('CMA - ONA (ng/m³)')